In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# tensorflow
import tensorflow as tf
from tensorflow import keras

from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Conv1D, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"

In [ ]:
df = pd.read_excel("data/Smart Freeways Data 2018 - Ramps All Days.xlsx",
                   skiprows=3, header=0)
df.head()

In [ ]:
df = df[['Row Labels', '27 Aug 18']]
df['Time'] = df['Row Labels']
df = df[['Row Labels', 'Time', '27 Aug 18']]
df.head()

In [ ]:
# For Row Labels, replace time format %H:%M - %H:%M with NaN
# For DateTime, replace everything else as NaN

df['Row Labels'] = df['Row Labels'].replace(
    '([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?\s-\s([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?',
    np.nan, regex=True)
df['Row Labels'] = df['Row Labels'].ffill(axis=0)
df['Row Labels'].unique()

- if *Row Labels* contain '[K][w][N]\s', drop row

In [ ]:
df = df[~df['Row Labels'].str.contains('KwN ', na=False)]
df = df[~df['Row Labels'].str.contains('Grand Total', na=False)]
df['Row Labels'].unique()

In [ ]:
df

In [ ]:
OR_df = df.copy()

In [ ]:
OR_df = OR_df[~OR_df['Row Labels'].str.contains('XR', na=False)]
OR_df = OR_df[~OR_df['Row Labels'].str.contains('210', na=False)]
OR_df = OR_df[OR_df['27 Aug 18'].notnull()]

OR_df.Time = OR_df.Time.str[:5]
OR_df = OR_df[OR_df.Time.str.contains(':', na=False)]

In [ ]:
OR_df = OR_df[~OR_df['Row Labels'].str.contains('T', na=False)]

In [ ]:
OR_df['Row Labels'].value_counts().sort_index()

In [ ]:
OR_df.Time.value_counts()

In [ ]:
OR_df['Time'] = pd.to_datetime(OR_df['Time'], format='%H:%M')
OR_df = OR_df.melt(id_vars=['Row Labels', 'Time'],
                  var_name='DateTime', value_name='Volume')
OR_df.DateTime = (pd.to_datetime(OR_df['DateTime']) +  
              pd.to_timedelta(OR_df.pop('Time').dt.strftime('%H:%M:%S')))

OR_df = OR_df.sort_values(['Row Labels', 'DateTime'], 
                          ascending=[True, True])
OR_df.reset_index(drop=True, inplace=True)
OR_df

In [ ]:
new = OR_df[(OR_df.DateTime >= '2018-08-27 8:30:00') &
     (OR_df.DateTime <= '2018-08-27 13:30:00')]

In [ ]:
fig, ax = plt.subplots()
p = sns.lineplot(data=new, x='DateTime', y='Volume', hue='Row Labels', ax=ax)
ax.legend().set_title('')
plt.show()

In [ ]:
res = pd.pivot_table(new, values='Volume', index='DateTime',
               columns='Row Labels').reset_index()

In [ ]:
#res.to_csv('data/OR_df.csv', index=False)

In [ ]:
g = res.set_index('DateTime')

In [ ]:
g['0084KWN-ORL'] = g['0084KWN-OR1L'] + g['0084KWN-OR2L']
g['0089KWN-ORL'] = g['0089KWN-OR1L'] + g['0089KWN-OR2L']
g = g[['0081KWN-OR1L', '0084KWN-ORL', '0089KWN-ORL', '0130KWN-OR1',
       '0150KWN-OR1', '0160KWN-OR1L', '0200KWN-OR1L', '0702KWN-OR1L']]
g.columns = ['H500', 'H549', 'H617', 'H547', 'H553', 'H554', 'H622', 'H558']
g

In [ ]:
g = g*4; g

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'lines.linewidth': 2})
fig, ax = plt.subplots(figsize=(10,5))
p = sns.lineplot(data=g, ax=ax, linewidth = 2)
ax.legend(loc='upper right', 
               ncol=round(len(g.columns)/2)).set_title('')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30])) 
ax.set_ylabel('Volume (veh/hr)', fontsize=20)
ax.set_xlabel('Time (HH:MM)', fontsize=18)
ax.grid()
#plt.savefig('fig/Ramps_Volume', bbox_inches="tight")
plt.show()

In [ ]:
g.to_csv('data/OR_df.csv', index=False)